# Классификация кредитных рейтингов

## Словесное описание задачи

На основе сведений о клиенте банка требуется определить его кредитный рейтинг. Предоставленные данные:
- `ID`: уникальный идентификатор записи
- `Customer_ID`: уникальный идентификатор клиента
- `Month`: месяц года, в рамках которого рассматриваются данные
- `Name`: имя клиента
- `Age`: возраст клиента
- `SSN`: номер социального страхования (social security number)
- `Occupation`: род деятельности
- `Annual_Income`: годовой доход
- `Monthly_Inhand_Salary`: ежемесячный доход
- `Num_Bank_Accounts`: число открытых банковских счетов клиента
- `Num_Credit_Card`: число других (взятых в других банках) кредитных карт клиента
- `Interest_Rate`: процентная ставка по кредитной карте
- `Num_of_Loan`: число взятых кредитов
- `Type_of_Loan`: типы взятых кредитов
- `Delay_from_due_date`: среднее число дней, через которые выплачивался просроченный платеж
- `Num_of_Delayed_Payment`: среднее число просроченных платежей
- `Changed_Credit_Limit`: процентное изменение лимита по кредитной карте
- `Num_Credit_Inquiries`: число запросов кредитного рейтинга
- `Credit_Mix`: оценка сбалансированности кредитных обязательств клиента
- `Outstanding_Debt`: остаток долга к уплате
- `Credit_Utilization_Ratio`: доля использованного баланса на кредитной карте
- `Credit_History_Age`: срок ведения кредитной истории
- `Payment_of_Min_Amount`: был ли выплачен только минимальный платёж по кредитной карте
- `Total_EMI_per_month`: ежемесячный платёж
- `Amount_invested_monthly`: сумма денег, ежемесячно инвестируемая
- `Payment_Behaviour`: характер платежей клиента
- `Monthly_Balance`: остаток денег на счетах клиента

## Чтение данных

Установим и импортируем необходимые для работы библиотеки.

In [ ]:
%pip install numpy matplotlib pandas scipy scikit-learn seaborn

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as stats
from sklearn.model_selection import KFold
import seaborn as sns
%matplotlib inline

In [ ]:
df = pd.read_csv('./train.csv')
df.head()

## Визуализация данных и основные характеристики

In [ ]:
df.describe().T

In [ ]:
from pandas.plotting import scatter_matrix
scatter_matrix(df, alpha=0.01, figsize=(16, 10))
pass

## Обработка пропущенных значений и выбросов

In [ ]:
df.isnull().sum()

Особенностью рассматриваемого датасета является характер содержащихся в нём данных: информация о всяком клиенте предоставлена по каждому месяцу, с января по август. Это позволяет более разумно заполнять пропущенные значения или исправлять выбросы --- средние значения и моды будут браться не по всей выборке, а по каждому из клиентов.

In [ ]:
df_clean = df.copy()

### Вспомогательные функции

In [ ]:
def get_group_min_max(df, groupby, column):
    """
    Группирует предоставленный df по groupby, затем находит
    во всякой группе моду, а затем возвращает наименьшее и
    наибольшее значение мод. Используется для оценки истинных
    границ значений по столбцу.
    """

    cur = df[df[column].notna()].groupby(groupby)[column].apply(list)
    x, y = cur.apply(lambda x: stats.mode(x)).apply([min, max])
    return x[0], y[0]


def age_to_months(string):
    """
    Переводит словесное описание периода времени (лет + месяцев) в
    эквивалентное численное значение месяцев
    """

    if pd.isna(string):
        return pd.NA
    years, months = map(int, [string.split()[0], string.split()[-2]])
    return years * 12 + months

### Нерелевантные столбцы

Из выборки удалены столбцы "ID", "Name" и "SSN", так как они не представляют никакой ценности в расчётах кредитного рейтинга.

In [ ]:
df_clean = df_clean.drop(columns=["ID", "Name", "SSN"])

### `Age`

In [ ]:
df_clean['Age'] = df_clean['Age'].str.rstrip('_').astype('int64')
mini, maxi = get_group_min_max(df_clean, 'Customer_ID', 'Age')
df_clean['Age'] = df_clean['Age'].transform(lambda x: x if mini - 1 <= x <= maxi + 1 else pd.NA)
df_clean['Age'] = df_clean.groupby('Customer_ID')['Age'].transform(lambda x: x.fillna(x.mode()[0]))

### `Occupation`

In [ ]:
df_clean['Occupation'] = df_clean['Occupation'].replace('_______', pd.NA)
df_clean['Occupation'] = df_clean.groupby('Customer_ID')['Occupation'].transform(lambda x: x.fillna(x.mode()[0]))

### `Annual_Income`

In [ ]:
df_clean['Annual_Income'] = df_clean['Annual_Income'].str.rstrip('_').astype('float64')
median = df_clean['Annual_Income'].median()
mad = np.median(np.abs(df_clean['Annual_Income'] - median))
z_scores = 0.6745 * (df_clean['Annual_Income'] - median) / mad
outliers = np.abs(z_scores) > 5
def replace_outliers(x):
    m = x[~outliers[x.index]].mean()
    x.loc[outliers[x.index]] = m
    return x
df_clean['Annual_Income'] = df_clean.groupby('Customer_ID')['Annual_Income'].transform(replace_outliers)

### `Monthly_Inhand_Salary`

In [ ]:
df_clean['Monthly_Inhand_Salary'] = df_clean.groupby('Customer_ID')['Monthly_Inhand_Salary'].transform(lambda x: x.fillna(x.mean()))

### `Num_Bank_Accounts`

In [ ]:
mini, maxi = get_group_min_max(df_clean, 'Customer_ID', 'Num_Bank_Accounts')
df_clean['Num_Bank_Accounts'] = df_clean['Num_Bank_Accounts'].transform(lambda x: x if mini - 1 <= x <= maxi + 1 else pd.NA)
df_clean['Num_Bank_Accounts'] = df_clean.groupby('Customer_ID')['Num_Bank_Accounts'].transform(lambda x: x.fillna(x.mode()[0]))

### `Num_Credit_Card`

In [ ]:
mini, maxi = get_group_min_max(df_clean, 'Customer_ID', 'Num_Credit_Card')
df_clean['Num_Credit_Card'] = df_clean['Num_Credit_Card'].transform(lambda x: x if mini - 1 <= x <= maxi + 1 else pd.NA)
df_clean['Num_Credit_Card'] = df_clean.groupby('Customer_ID')['Num_Credit_Card'].transform(lambda x: x.fillna(x.mode()[0]))

### `Interest_Rate`

In [ ]:
mini, maxi = get_group_min_max(df_clean, 'Customer_ID', 'Interest_Rate')
df_clean['Interest_Rate'] = df_clean['Interest_Rate'].transform(lambda x: x if mini - 1 <= x <= maxi + 1 else pd.NA)
df_clean['Interest_Rate'] = df_clean.groupby('Customer_ID')['Interest_Rate'].transform(lambda x: x.fillna(x.mode()[0]))

### `Num_of_Loan`

In [ ]:
df_clean['Num_of_Loan'] = df_clean['Num_of_Loan'].str.rstrip('_').astype('int64')
mini, maxi = get_group_min_max(df_clean, 'Customer_ID', 'Num_of_Loan')
df_clean['Num_of_Loan'] = df_clean['Num_of_Loan'].transform(lambda x: x if mini - 1 <= x <= maxi + 1 else pd.NA)
df_clean['Num_of_Loan'] = df_clean.groupby('Customer_ID')['Num_of_Loan'].transform(lambda x: x.fillna(x.mode()[0]))

### `Num_of_Delayed_Payment`

In [ ]:
df_clean['Num_of_Delayed_Payment'] = df_clean['Num_of_Delayed_Payment'].str.rstrip('_')
df_clean['Num_of_Delayed_Payment'] = df_clean.groupby('Customer_ID')['Num_of_Delayed_Payment'].transform(lambda x: x.fillna(x.mode()[0])).astype('int64')
mini, maxi = get_group_min_max(df_clean, 'Customer_ID', 'Num_of_Delayed_Payment')
df_clean['Num_of_Delayed_Payment'] = df_clean['Num_of_Delayed_Payment'].transform(lambda x: x if mini - 1 <= x <= maxi + 1 else pd.NA)
df_clean['Num_of_Delayed_Payment'] = df_clean.groupby('Customer_ID')['Num_of_Delayed_Payment'].transform(lambda x: x.fillna(x.mode()[0]))

### `Changed_Credit_Limit`

In [ ]:
df_clean['Changed_Credit_Limit'] = pd.to_numeric(df_clean['Changed_Credit_Limit'], errors='coerce')
df_clean['Changed_Credit_Limit'] = df_clean.groupby('Customer_ID')['Changed_Credit_Limit'].transform(lambda x: x.fillna(x.mean()))

### `Num_Credit_Inquiries`

In [ ]:
df_clean['Num_Credit_Inquiries'] = df_clean.groupby('Customer_ID')['Num_Credit_Inquiries'].transform(lambda x: x.fillna(x.mode()[0])).astype('int64')
mini, maxi = get_group_min_max(df_clean, 'Customer_ID', 'Num_Credit_Inquiries')
df_clean['Num_Credit_Inquiries'] = df_clean['Num_Credit_Inquiries'].transform(lambda x: x if mini - 1 <= x <= maxi + 1 else pd.NA)
df_clean['Num_Credit_Inquiries'] = df_clean.groupby('Customer_ID')['Num_Credit_Inquiries'].transform(lambda x: x.fillna(x.mode()[0]))

### `Credit_Mix`

In [ ]:
df_clean['Credit_Mix'] = df_clean['Credit_Mix'].transform(lambda x: pd.NA if x == '_' else x)
df_clean['Credit_Mix'] = df_clean.groupby('Customer_ID')['Credit_Mix'].transform(lambda x: x.fillna(x.mode()[0]))

### `Outstanding_Debt`

In [ ]:
df_clean['Outstanding_Debt'] = df_clean['Outstanding_Debt'].str.rstrip('_').astype('float64')

### `Total_EMI_per_month`

In [ ]:
mini, maxi = get_group_min_max(df_clean, 'Customer_ID', 'Total_EMI_per_month')
df_clean['Total_EMI_per_month'] = df_clean['Total_EMI_per_month'].transform(lambda x: x if mini - 1 <= x <= maxi + 1 else pd.NA)
df_clean['Total_EMI_per_month'] = df_clean.groupby('Customer_ID')['Total_EMI_per_month'].transform(lambda x: x.fillna(x.mean()))

### `Amount_invested_monthly`

In [ ]:
df_clean['Amount_invested_monthly'] = pd.to_numeric(df_clean['Amount_invested_monthly'], errors='coerce')
df_clean['Amount_invested_monthly'] = df_clean.groupby('Customer_ID')['Amount_invested_monthly'].transform(lambda x: x.fillna(x.mean()))

### `Monthly_Balance`

In [ ]:
df_clean['Monthly_Balance'] = pd.to_numeric(df_clean['Monthly_Balance'], errors='coerce')
df_clean['Monthly_Balance'] = df_clean.groupby('Customer_ID')['Monthly_Balance'].transform(lambda x: x.fillna(x.mean()))

### `Payment_Behaviour`

In [ ]:
df_clean['Payment_Behaviour'] = df_clean['Payment_Behaviour'].transform(lambda x: pd.NA if x == '!@9#%8' else x)
df_clean['Payment_Behaviour'] = df_clean.groupby('Customer_ID')['Payment_Behaviour'].transform(lambda x: x.fillna(x.mode()[0]))

### `Credit_History_Age`

In [ ]:
df_clean['Credit_History_Age'] = pd.to_numeric(df_clean['Credit_History_Age'].transform(age_to_months), errors='coerce')
df_clean['Credit_History_Age'] = df_clean.groupby('Customer_ID')['Credit_History_Age'].transform(lambda x: x.interpolate('linear', limit_direction='both').round())

### `Type_of_Loan`

In [ ]:
df_clean['Type_of_Loan'] = df_clean['Type_of_Loan'].apply(lambda x: x.lower().replace('and ', '').replace(', ', ',').strip() if pd.notna(x) else x)
df_clean['Type_of_Loan'] = df_clean['Type_of_Loan'].replace([pd.NA], 'no loan')

### Итого

После обработки значений столбец `Customer_ID` может быть убран, так как никакой ценности для решения задачи он не несёт.

In [ ]:
df_clean = df_clean.drop(columns=["Customer_ID"])

In [ ]:
df_clean.head()

In [ ]:
df_clean.describe().T

In [ ]:
plt.figure(figsize=(16, 10))
sns.heatmap(df_clean.corr(numeric_only=True), square=True, annot=True, cmap="PiYG")

## Нормализация

In [ ]:
df_numeric = df_clean.copy()

In [ ]:
numeric_columns = df_numeric.select_dtypes(include=['int', 'float']).columns.to_list()
for column in numeric_columns:
    df_numeric[column] = (df_numeric[column] - df_numeric[column].mean()) / df_numeric[column].std()

## Обработка категориальных признаков

### `Month`

In [ ]:
df_numeric.groupby('Month').size()

In [ ]:
df_numeric = df_numeric.replace(
    {
        'Month': {
            'January': 0,
            'February': 1,
            'March': 2,
            'April': 3,
            'May': 4,
            'June': 5,
            'July': 6,
            'August': 7,
        }
    }
)
df_numeric['Month'] = (df_numeric['Month'] - df_numeric['Month'].mean()) / df_numeric['Month'].std()

### `Credit_Score`

In [ ]:
df_numeric.groupby('Credit_Score').size()

In [ ]:
df_numeric = df_numeric.replace(
    {
        'Credit_Score': {
            'Poor': 0,
            'Standard': 1,
            'Good': 2,
        }
    }
)

### `Occupation`

In [ ]:
df_numeric.groupby('Occupation').size()

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
df_numeric['Occupation_enc'] = 0

for train_idx, valid_idx in kf.split(df_numeric):
    train, valid = df_numeric.iloc[train_idx], df_numeric.iloc[valid_idx]
    means = train.groupby('Occupation')['Credit_Score'].mean()
    df_numeric.loc[valid_idx, 'Occupation_enc'] = valid['Occupation'].map(means).fillna(df_numeric['Credit_Score'].mean())

df_numeric['Occupation_enc'] += np.random.normal(0, 0.001, df_numeric['Occupation_enc'].shape)
df_numeric['Occupation'] = df_numeric['Occupation_enc']
df_numeric = df_numeric.drop(columns=['Occupation_enc'])

### `Type_of_Loan`

In [ ]:
df_numeric.groupby('Type_of_Loan').size()

In [ ]:
loan_types = set()
df_numeric['Type_of_Loan'].str.split(',').apply(loan_types.update)

for loan in loan_types:
    df_numeric[loan.replace(' ', '_')] = df_numeric['Type_of_Loan'].transform(lambda x: 1 if loan in x.split(',') else 0)

df_numeric = df_numeric.drop(columns=['Type_of_Loan'])

### `Credit_Mix`

In [ ]:
df_numeric.groupby('Credit_Mix').size()

In [ ]:
df_numeric = df_numeric.replace(
    {
        'Credit_Mix': {
            'Bad': 0,
            'Standard': 1,
            'Good': 2,
        }
    }
)

### `Payment_of_Min_Amount`

In [ ]:
df_numeric.groupby('Payment_of_Min_Amount').size()

In [ ]:
df_numeric = df_numeric.replace(
    {
        'Payment_of_Min_Amount': {
            'No': 0,
            'NM': 1,
            'Yes': 2,
        }
    }
)

### `Payment_Behaviour`

In [ ]:
df_numeric.groupby('Payment_Behaviour').size()

In [ ]:
split = df_numeric['Payment_Behaviour'].str.split('_', expand=True)
df_numeric['Payment_Behaviour_Spent'] = split[0]
df_numeric['Payment_Behaviour_Payments'] = split[2]

df_numeric = df_numeric.replace(
    {
        'Payment_Behaviour_Spent': {
            'Low': 0,
            'High': 1,
        },
        'Payment_Behaviour_Payments': {
            'Small': 0,
            'Medium': 1,
            'Large': 2,
        }
    }
)
df_numeric = df_numeric.drop(columns=['Payment_Behaviour'])

## Разбиение на обучающую и тестовую выборки

In [ ]:
from sklearn.model_selection import train_test_split

X = df_numeric.drop('Credit_Score', axis=1).to_numpy()
y = df_numeric['Credit_Score'].to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

## Обучение

### kNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=8)
knn.fit(X_train, y_train)
knn_y_pred = knn.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_test, knn_y_pred))

In [ ]:
print(confusion_matrix(y_test, knn_y_pred))

Ошибки на обучающей и тестовой выборках:

In [ ]:
print(f'Обучающая: {1 - knn.score(X_train, y_train)}')
print(f'Тестовая: {1 - knn.score(X_test, y_test)}')

#### Подбор оптимального `k`

In [ ]:
from sklearn.model_selection import cross_val_score

k_values = list(range(1, 21))
cv_scores = [ cross_val_score(KNeighborsClassifier(n_neighbors=k), X_train, y_train, scoring='accuracy').mean() for k in k_values ]

plt.plot(k_values, cv_scores, marker='o')
plt.xlabel('Значение k')
plt.ylabel('Кросс-валидация, точность')
plt.title('Выбор k')
plt.show()

Рассмотрим работу при $k = 3$:

In [ ]:
opt_knn = KNeighborsClassifier(n_neighbors=3)
opt_knn.fit(X_train, y_train)
opt_knn_y_pred = opt_knn.predict(X_test)

In [ ]:
print(classification_report(y_test, opt_knn_y_pred))

In [ ]:
print(confusion_matrix(y_test, opt_knn_y_pred))

Ошибки на обучающей и тестовой выборках:

In [ ]:
print(f'Обучающая: {1 - opt_knn.score(X_train, y_train)}')
print(f'Тестовая: {1 - opt_knn.score(X_test, y_test)}')

### `RandomForestClassifier`

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(random_state=42)
rfc.fit(X_train, y_train)
rfc_y_pred = rfc.predict(X_test)

In [ ]:
print(classification_report(y_test, rfc_y_pred))

In [ ]:
print(confusion_matrix(y_test, rfc_y_pred))

Ошибки на обучающей и тестовой выборках:

In [ ]:
print(f'Обучающая: {1 - rfc.score(X_train, y_train)}')
print(f'Тестовая: {1 - rfc.score(X_test, y_test)}')

### `GradientBoostingClassifier`

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gbc = GradientBoostingClassifier(random_state=42)
gbc.fit(X_train, y_train)
gbc_y_pred = gbc.predict(X_test)

In [ ]:
print(classification_report(y_test, gbc_y_pred))
print(confusion_matrix(y_test, gbc_y_pred))

Ошибки на обучающей и тестовой выборках:

In [ ]:
print(f'Обучающая: {1 - gbc.score(X_train, y_train)}')
print(f'Тестовая: {1 - gbc.score(X_test, y_test)}')

## Выводы

По результатам работы наилучшей точности удалось добиться при использовании `RandomForestClassifier`, со значением $0.82$.

Выбор $k = 3$ в методе kNN даёт баланс точности работы и обобщённости модели, позволяя достичь точности $0.75$.

Метод `GradientBoostingClassifier` показал наихудший результат точности со значением $0.71$.